### This code is written to investigate how the cost of living in different countries correlates with the actual quality of life.

In [1]:
# import packages
import pandas as pd
import plotly.express as px

In [2]:
# read data
rank = pd.read_csv('input/rank.csv', sep='\t')
reg = pd.read_csv('input/regions.csv', sep='\t')
mrg = pd.merge(rank, reg, on='Country')
mrg.head(2)
# write merged data to file for manual check
mrg.to_excel('results/merged.xlsx', index=False)

In [3]:
# read custom correct data
mrg = pd.read_excel('results/merged_m.xlsx')
mrg.head(2)

,Rank,Country,Quality_of_Life_Index,Purchasing_Power_Index,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Property_Price_to_Income_Ratio,Traffic_Commute_Time_Index,Pollution_Index,Climate_Index,Region
0,1,Netherlands,196.7,107.8,72.7,77.1,68.6,7.2,25.5,23.7,86.8,Western_Europe
1,2,Denmark,194.7,105.0,73.4,79.2,78.6,6.6,28.5,21.3,81.8,Northern_Europe


In [4]:
# define colors for regions
region_colors = {'Oceania': '#f4a259',
                 'Western_Asia': '#F44336',
                 'Northern_Europe': '#9ed0ab',
                 'Northern_America': '#4b6d8f',
                 'South-eastern_Asia': '#E57373',
                 'Australia': '#aa79d1',
                 'Central_America': '#6d8dc3',
                 'Southern_Asia': '#f9d6b8',
                 'Eastern_Asia': '#EF9A9A',
                 'Southern_Europe': '#bde3c5',
                 'Eastern_Europe': '#d0f0dc',
                 'South_America': '#6d8dc3',
                 'Northern_Africa': '#f9d9a0',
                 'Eastern_Africa': '#f7c36d',
                 'Western_Europe': '#7fcd91',
                 'Southern_Africa': '#e5981d',
                 'Western_Africa': '#edb880',
                 'Central_Asia': '#FFCDD2'}

## Quality_of_Life_Index


In [5]:
# Create interactive scatterplot using Plotly Express
custom_order = ['Western_Europe', 'Northern_Europe', 'Eastern_Europe', 'Southern_Europe',
                'Western_Asia', 'Central_Asia', 'Eastern_Asia', 'Southern_Asia',
                'Northern_Africa', 'Western_Africa', 'Eastern_Africa', 'Southern_Africa',
                'South_America', 'Central_America', 'South-eastern_Asia', 'Australia',
                'Northern_America']

fig = px.scatter(mrg, x='Quality_of_Life_Index', y='Cost_of_Living_Index', color='Region',
                 color_discrete_map=region_colors, symbol='Region', hover_name='Country',
                 width=800, height=600, category_orders={'Region': ['Australia', 'Central_America', 'Northern_America', 'South_America',
                                             'Eastern_Africa', 'Northern_Africa', 'Southern_Africa', 'Western_Africa',
                                             'Eastern_Asia', 'Southern_Asia', 'South-eastern_Asia', 'Western_Asia','Central_Asia',
                                             'Eastern_Europe', 'Northern_Europe', 'Southern_Europe', 'Western_Europe']})

# Modify the legend and layout
fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='Gray')),
                  selector=dict(mode='markers'))
fig.update_layout(legend=dict(title='', orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02),
                  font=dict(family='Arial', size=14),
                  margin=dict(l=40, r=40, t=20, b=20), 
                  xaxis_title='Quality of Life Index', yaxis_title='Cost of Living Index',
                  width=900, height=600, template='plotly_white')

import plotly.io as pio
pio.write_html(fig, file='results/Cost_vs_quality.html', auto_open=True)

Quality of Life Index (higher is better) is an estimation of overall quality of life by using an empirical formula which takes into account 
- purchasing power index (higher is better), 
- pollution index (lower is better), 
- house price to income ratio (lower is better), 
- cost of living index (lower is better), 
- safety index (higher is better), 
- health care index (higher is better), 
- traffic commute time index (lower is better) and 
- climate index (higher is better).

These indices are relative to New York City (NYC). Which means that for New York City, each index should be 100(%). If a city has a Cost of Living Index of 120, it means Numbeo has estimated it is 20% more expensive than New York (excluding rent).

## Which countries has the price-to-cost ratio lower or higher than average?

In [6]:
# Add to data information about Cost to quality ratio and quantiles
mrg['Cost_vs_quality_ratio'] = mrg['Quality_of_Life_Index']/mrg['Cost_of_Living_Index']
mrg['quantiles'] = pd.qcut(mrg['Cost_vs_quality_ratio'], q=4, labels=['bad_high','bad_low','good_low', 'good_high'])

mrg.head()

,Rank,Country,Quality_of_Life_Index,Purchasing_Power_Index,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Property_Price_to_Income_Ratio,Traffic_Commute_Time_Index,Pollution_Index,Climate_Index,Region,Cost_vs_quality_ratio,quantiles
0,1,Netherlands,196.7,107.8,72.7,77.1,68.6,7.2,25.5,23.7,86.8,Western_Europe,2.867347,good_low
1,2,Denmark,194.7,105.0,73.4,79.2,78.6,6.6,28.5,21.3,81.8,Northern_Europe,2.477099,bad_low
2,3,Switzerland,193.6,118.7,76.4,74.5,114.2,9.7,28.4,19.6,79.6,Western_Europe,1.695271,bad_high
3,4,Luxembourg,192.9,127.1,64.9,75.0,73.2,11.5,29.1,21.9,82.6,Western_Europe,2.635246,bad_low
4,5,Finland,190.5,98.8,73.2,77.1,67.5,7.8,27.2,12.0,59.2,Northern_Europe,2.822222,good_low


In [7]:
# Visualize quantiles information
quantiles_colors = {'good_high': '#759116',
                 'good_low': '#056517',
                 'bad_low': '#f57a9b',
                 'bad_high': '#de1a24'}

fig = px.scatter(mrg, x='Quality_of_Life_Index', y='Cost_of_Living_Index', color='quantiles',
                 symbol='quantiles', hover_name='Country', color_discrete_map=quantiles_colors,
                 category_orders={'quantiles': ['good_high', 'good_low','bad_low','bad_high']},
                 width=800, height=600)

# Modify the legend and layout
fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='Gray')),
                  selector=dict(mode='markers'))
fig.update_layout(legend=dict(title='', orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02),
                  font=dict(family='Arial', size=14),
                  margin=dict(l=40, r=40, t=20, b=20), 
                  xaxis_title='Quality of Life Index', yaxis_title='Cost of Living Index',
                  width=900, height=600, template='plotly_white')

pio.write_html(fig, file='results/Quantiles.html', auto_open=True)

## Choose top countries with hight life quality index and good Cost to quality ratio 

In [8]:
mrg1 = mrg[(mrg['Quality_of_Life_Index'] >= 160) & 
           ((mrg['quantiles'] == 'good_high') | (mrg['quantiles'] == 'good_low'))]

mrg1.to_excel('results/top_counties.xlsx')
mrg1

,Rank,Country,Quality_of_Life_Index,Purchasing_Power_Index,Safety_Index,Health_Care_Index,Cost_of_Living_Index,Property_Price_to_Income_Ratio,Traffic_Commute_Time_Index,Pollution_Index,Climate_Index,Region,Cost_vs_quality_ratio,quantiles
0,1,Netherlands,196.7,107.8,72.7,77.1,68.6,7.2,25.5,23.7,86.8,Western_Europe,2.867347,good_low
4,5,Finland,190.5,98.8,73.2,77.1,67.5,7.8,27.2,12.0,59.2,Northern_Europe,2.822222,good_low
7,8,Oman,184.7,108.2,80.3,58.2,47.8,3.6,22.6,37.2,67.2,Western_Asia,3.864017,good_high
10,11,Germany,179.0,107.6,62.4,71.9,62.9,10.7,31.2,28.8,83.3,Western_Europe,2.845787,good_low
14,15,United Arab Emirates,175.7,123.4,84.9,69.6,60.3,3.0,36.1,48.2,45.8,Western_Asia,2.913765,good_low
15,16,Spain,173.8,82.5,64.3,77.9,50.6,7.8,29.1,39.6,93.7,Southern_Europe,3.434783,good_high
16,18,Estonia,171.9,61.4,74.9,74.1,54.8,11.8,23.8,17.9,64.3,Northern_Europe,3.136861,good_low
17,19,Slovenia,169.3,61.7,75.8,65.3,50.8,12.1,26.1,22.7,77.6,Southern_Europe,3.332677,good_low
19,22,Portugal,163.8,51.7,68.5,71.5,45.3,12.6,29.5,30.6,97.8,Southern_Europe,3.615894,good_high
20,24,Croatia,163.2,56.2,75.4,63.9,46.7,13.2,26.5,32.0,89.7,Southern_Europe,3.494647,good_high


### These top countries have a good quality of life and at the same time a good cost to quality index. Further investigation at the city level is needed.